In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [2]:
from api_keys import g_key

In [3]:
city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,banikoara,11.2985,2.4386,77.41,77,69,4.47,BJ,1626731686
1,1,georgetown,5.4112,100.3354,80.89,88,20,1.99,MY,1626731507
2,2,constitucion,-35.3333,-72.4167,54.55,67,18,10.80,CL,1626731687
3,3,albany,42.6001,-73.9662,81.25,78,93,0.29,US,1626731687
4,4,zabid,14.1951,43.3152,87.85,57,100,12.77,YE,1626731688


In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,aripuana,-9.1667,-60.6333,79.90,48,0,1.79,BR,1626731690
17,17,mus,39.0000,41.7500,71.78,29,0,2.06,TR,1626731693
40,40,shahrud,36.4182,54.9763,73.29,45,0,7.18,IR,1626731701
211,211,matviyivka,47.0747,35.1444,78.80,55,0,6.55,UA,1626731767
225,225,bykovo,55.6361,38.0803,70.56,83,0,6.71,RU,1626731772
234,234,kumburgaz,41.0211,28.4772,78.71,89,0,4.14,TR,1626731776
252,252,pimentel,-6.8367,-79.9342,73.69,80,0,5.01,PE,1626731782
271,271,ushtobe,45.2514,77.9838,72.61,25,0,3.15,KZ,1626731789
315,315,newport,51.5877,-2.9984,73.69,77,0,3.60,GB,1626731806
343,343,seoul,37.5683,126.9778,76.44,94,0,3.44,KR,1626731588


In [7]:
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
12,aripuana,BR,-9.1667,-60.6333,
17,mus,TR,39.0000,41.7500,
40,shahrud,IR,36.4182,54.9763,
211,matviyivka,UA,47.0747,35.1444,
225,bykovo,RU,55.6361,38.0803,
234,kumburgaz,TR,41.0211,28.4772,
252,pimentel,PE,-6.8367,-79.9342,
271,ushtobe,KZ,45.2514,77.9838,
315,newport,GB,51.5877,-2.9984,
343,seoul,KR,37.5683,126.9778,


In [8]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
12,aripuana,BR,-9.1667,-60.6333,
17,mus,TR,39.0000,41.7500,
40,shahrud,IR,36.4182,54.9763,
211,matviyivka,UA,47.0747,35.1444,
225,bykovo,RU,55.6361,38.0803,
234,kumburgaz,TR,41.0211,28.4772,
252,pimentel,PE,-6.8367,-79.9342,
271,ushtobe,KZ,45.2514,77.9838,
315,newport,GB,51.5877,-2.9984,
343,seoul,KR,37.5683,126.9778,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))